<a href="https://colab.research.google.com/github/Chhaya-Tundwal05/pg-sales-agent/blob/sample_agent_yash/PG_Sales_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import PredictionGuard
from predictionguard import PredictionGuard
from getpass import getpass

# Set your Prediction Guard token as an environmental variable.
os.environ["PREDICTIONGUARD_API_KEY"] = "u7ojcxj2jrQ9ygro7sRCXETHfgQf84NmhnXUFEsk"

client = PredictionGuard()

os.environ['SERPAPI_API_KEY'] = "fb52c594eea49307272b7536ee61eadd88cf8a7199de7eeba8e3af82d8f44a11"




ModuleNotFoundError: Module langchain_community.agent_toolkits.load_tools not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`

In [6]:
pip install langchain-predictionguard

In [ ]:
llm = PredictionGuard(
    model= "Hermes-3-Llama-3.1-70B",
    predictionguard_api_key=os.environ["PREDICTIONGUARD_API_KEY"],
    temperature=0.75,
    max_tokens=100,
    stop=["000"]  # Stop generation at "000" or double newlines
)

In [9]:
import os
import pandas as pd
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_predictionguard import PredictionGuard  # Corrected import

# Set your Prediction Guard API key as an environmental variable
os.environ["PREDICTIONGUARD_API_KEY"] = "u7ojcxj2jrQ9ygro7sRCXETHfgQf84NmhnXUFEsk"

# Initialize Prediction Guard LLM (corrected)
#llm = PredictionGuard(model="Hermes-2-Pro-Llama-3-8B")  # Choose the correct model
llm = PredictionGuard(
    model= "Hermes-3-Llama-3.1-70B",
    predictionguard_api_key=os.environ["PREDICTIONGUARD_API_KEY"],
    temperature=0.75,
    max_tokens=100,
    stop=["000"]  # Stop generation at "000" or double newlines
)

# Load the dataset
df = pd.read_csv("Dataset_with_Flowing_Conversations.csv")

# Define a prompt template for summarization (fixed variable mismatch)
prompt_template = PromptTemplate(
    input_variables=["notes"],  # Corrected to match the placeholder
    template="Summarize the following meeting notes in 2-3 sentences, focusing on key points, decisions, and action items:\n\n{notes}"
)

# Create a summarization chain
summarization_chain = LLMChain(llm=llm, prompt=prompt_template)

# Function to summarize notes using LangChain and Prediction Guard
def summarize_notes(notes):
    try:
        if not isinstance(notes, str) or notes.strip() == "":
            return "No valid notes available."

        summary = summarization_chain.run({"notes": notes})  # Fixed function call
        return summary.strip() if summary else "Summarization failed."
    except Exception as e:
        return f"Error: {str(e)}"

# Apply summarization to the dataset
df["Summary"] = df["Meeting Notes"].apply(summarize_notes)

# Save the updated dataset
df.to_csv("summarized_leads.csv", index=False)

# Print a sample of the summarized data
print(df[["Name", "Company", "Summary"]].head())

# Function to deduce next steps based on summarized notes
def deduce_next_steps(row):
    summary = row["Summary"]
    if "interested" in summary.lower() and "budget" in summary.lower():
        return "Schedule a follow-up meeting to discuss pricing and contract details."
    elif "proposal" in summary.lower():
        return "Send a detailed proposal and schedule a call to address questions."
    elif "demo" in summary.lower():
        return "Arrange a product demo and provide additional use cases."
    else:
        return "Send a follow-up email with additional information and case studies."

# Apply next steps
df["Next Steps"] = df.apply(deduce_next_steps, axis=1)

# Save the final dataset
df.to_csv("final_leads.csv", index=False)

<ipython-input-9-567184082d01>:30: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summarization_chain = LLMChain(llm=llm, prompt=prompt_template)
<ipython-input-9-567184082d01>:38: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = summarization_chain.run({"notes": notes})  # Fixed function call


                  Name                        Company  \
0         Allison Hill                     Taylor Inc   
1          Carla Kelly         Adams, Zuniga and Wong   
2    Patricia Peterson                  Graham-Chavez   
3       Cindy Anderson  Johnston, Sanchez and Kennedy   
4  Dr. Tracy House DVM                  Russell Group   

                                             Summary  
0  The engineer inquired about integrating Predic...  
1  The CEO of an organization is looking for a sc...  
2  The CIO inquired about Prediction Guard's comp...  
3  The CIO inquired about Prediction Guard's comp...  
4  - Lead shares details about their data infrast...  
